In [1]:

%pprint

Pretty printing has been turned OFF


In [2]:

import sys

# Insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../py')

In [3]:

%%time
from pandas import DataFrame

from storage import Storage
s = Storage()

from ha_utils import HeaderAnalysis
ha = HeaderAnalysis(verbose=False)

from scrape_utils import WebScrapingUtilities
wsu = WebScrapingUtilities()
uri = wsu.secrets_json['neo4j']['connect_url']
user =  wsu.secrets_json['neo4j']['username']
password = wsu.secrets_json['neo4j']['password']

from cypher_utils import CypherUtilities
cu = CypherUtilities(uri=uri, user=user, password=password, driver=None, s=s, ha=ha)

from hc_utils import HeaderCategories
hc = HeaderCategories(cu=cu, verbose=False)

from section_utils import SectionUtilities
su = SectionUtilities(s=s, ha=ha, cu=cu, verbose=False)

from lr_utils import LrUtilities
lru = LrUtilities(ha=ha, cu=cu, hc=hc, verbose=False)
lru.build_isheader_logistic_regression_elements()
lru.build_pos_logistic_regression_elements()

import warnings
warnings.filterwarnings('ignore')

CPU times: total: 49.4 s
Wall time: 54.4 s


In [4]:

%run ../load_magic/dataframes.py
from datetime import datetime
from urllib.parse import urlparse, parse_qs
from urllib.error import HTTPError
import random
import requests

In [5]:

cookies_dict = {
    'ht': '{"quantcast":["D"]}',
    'gdId': '2b24e54b-29d2-46bd-9732-f6aef00c084e',
    'trs': 'direct:direct:direct:2022-04-04+03:20:19.923:undefined:undefined',
    'gdGPCset': 'true',
    'uc': 'B938D722CF68A3271174359AF29F93FEAB3FE2BBFF32C5FEA98CA1F5BE33AF94595A2DB4532FE1BA7A17CDE7E5BCEFA6534500B4E60A3FBF1E619DB45EB262E468246B7DF0D375AB79736822ECA37FF37D8FEFC0DED2F8F7FC80CE38D7B86C6A2E69C4417CCD8C4BE3E41A838ABAB104B96511C509D2F0315076D82A6128CF512741ABD354EDAC8529C2648A447E254F',
    'G_ENABLED_IDPS': 'google',
    'optimizelyEndUserId': 'oeu1653407247171r0.1354439941740344',
    'fpvc': '2',
    'indeedCtk': '1g56t61oeputh802',
    '_optionalConsent': 'true',
    '_gcl_au': '1.1.1410759256.1654868358',
    '_rdt_uuid': '1654868358827.cf5bfcfe-723c-40d3-b43c-ce699a4e33ea',
    '__pdst': 'a6be13e6db3c419a8a1a3cc281931765',
    '_fbp': 'fb.1.1654868359076.687854775',
    '_tt_enable_cookie': '1',
    '_ttp': '33c711dd-4788-48e9-aee2-754e8912e7b4',
    'JSESSIONID_JX_APP': '46D5BC327C4A37CDF3BE11CA0179D799',
    'GSESSIONID': '2b24e54b-29d2-46bd-9732-f6aef00c084e+1655477750481',
    'cass': '0',
    'JSESSIONID': 'C0CAD9ADF3FC2D5E811BE1D86D3724CE',
    '_gid': 'GA1.2.597320307.1655477753',
    '__gads': 'ID',
    '__gpi': 'UID',
    '_pin_unauth': 'dWlkPVlqTTVNbU15WVdFdE1tVmtZeTAwT1RkbExUaGpaR1l0TWpVd1pUZGhZVEJpTVdRMQ',
    'gdsid': '1655477750481:1655479708711:583F7F1FCA59FDBC937B963C3CDEC6F7',
    'at': '_h_DdAiCui-I3GulpXX2k5WyAdlNza6Bfg1l1u_jyMKKhNewrHN6F0Uji6Xo6lIoVB0t3aQWq7SPELxsLrxJOubiP2sY70pBKKzpILoQd7BQv24GZL5ANODjo2rrHDzZPYfcJRqmVgTj8mnRS3XflnS4fCafbddFkXkEBZ1tRMceGpbUaBjQabP801lS9_ZLMZUkZmR2D795W0YOLP8blyEcUXHQID12vt1-mgbuyuopPR7SRCVrTZI7Dc4GPWKmzks171b-05DWJaTmimU5vuVNz3Ev5iBcupKbTUooazW1fTrIB-ks2CG-Q2DzaBx1A2DfHSep8HA0yndndsRvHo_n2XZ6vu96HphoYF9VcuPxo1WUErR7VwU-FgHQKqiOMNC1JmhHp7scxf_ePqS2G4BSbqF7MU9Rvwl2GI1ryLgJMWen-HTm38fMN5xgFu1FQZe1e7S59y8PF6YpfoGHh1f_m8k-2N1S8Gi_0Mf1RGePsYraKif2mDz-08XmCj7OKXLUeM8HwnemcqB_1_miEQiL4j7AudNAINUM4iva9riAejwcIrs_7E7dwN_5moCrqjlWlx6YB4f56t-LK4Splum3fOKW47wN6d_8uG4rfsdwgVHuMwNFVQOYNGeEX024qaFZxEzyJBQx3UwT6Y7dOm7THqK7bbWq3WKCl4VN3SieRCs8r_aB_HVhq-0TAvqeebXUsEkU8z4iYrUbrf7SR3rS668UBmy4RaUkCS04gZkQxO-krJ4qMs6Bt_6laVX_pI6pMzgWTvAumM3YA0OzTCQYzUVQIkXrKzZz-hxKeIWKWjc6sSCmchfzH1-0tNFaVRPIv1Gmf2ELM8OIadRwjnXJQ9y4ppVj6QmrJMg9whXvJCSYDskqywXzXLcQnFt-',
    'asst': '1655479708.0',
    '__cf_bm': 'vIfYrUEKWDHLlFftluvvnQtKgLRWZEArM_cqPRwz_zI-1655479708-0-AXpWnquAMSkVzILLGwlZGWP7KdlrtP6g2B5vhV+MZw4oqFnvgFY/JIpoVnTgG0D7zB+ozciM9ZAxLrWClWh3LxQ',
    '_ga_RJF0GNZNXE': 'GS1.1.1655479710.3.0.1655479710.60',
    '_ga': 'GA1.2.6897551.1654868357',
    '_GA_Job_Session': 'true',
    'amp_a46b8b': 't8Dx3Eyg-zlel9fAUiSVKF.NTUyNDY5NjA',
    'OptanonConsent': 'isGpcEnabled',
    'AWSALB': 'uWCnnNFilbpBc8zqg81c8WfHhvnkYLQn2hI2FqPMtNlC8/A8lJv1Rj8AmjGOxsSQp5JvOXFzFESUIFvNCGIpZpKmanLUuAQpnoB7ZLZyQ/Fq9ec7XjBtLuL4iwfPrpZoYLzatLCHbBozUDU5jkcc0XX58owEVb5CsNX0zMXWYBpNDSyNiVpuGsudZc21Ag',
    'AWSALBCORS': 'uWCnnNFilbpBc8zqg81c8WfHhvnkYLQn2hI2FqPMtNlC8/A8lJv1Rj8AmjGOxsSQp5JvOXFzFESUIFvNCGIpZpKmanLUuAQpnoB7ZLZyQ/Fq9ec7XjBtLuL4iwfPrpZoYLzatLCHbBozUDU5jkcc0XX58owEVb5CsNX0zMXWYBpNDSyNiVpuGsudZc21Ag',
    'ki_t': '1655479712026;1655479712026;1655479712026;1;1',
    'ki_r': '',
    'ki_u': '1b5c216e-1907-87cb-aa9a-9f58',
    'ki_s': '184113:1.0.0.0.1',
    '_ga_RC95PMVB3H': 'GS1.1.1655479710.3.1.1655479717.53'
}
headers_dict = {'User-Agent': 'Mozilla/5.0'}


----

In [11]:

file_path = '../data/html/glassdoor_email.html'
page_soup = get_page_soup(file_path)
css_selector = 'a'
link_soups_list = page_soup.select(css_selector)

In [12]:

url_strs_list = []
rows_list = []
for link_soup in link_soups_list:
    url_str = link_soup['href']
    row_dict = {k: v[0] for k, v in parse_qs(urlparse(url_str).query).items()}
    row_dict['url_str'] = url_str
    rows_list.append(row_dict)
df = pd.DataFrame(rows_list)
mask_series = df.pos.isnull()
url_strs_list = df[~mask_series].url_str.tolist()

In [13]:

files_list = []
desc_regex = re.compile(r'"job":\{"description":"([^}]+)\}')
job_title_regex = re.compile(r'"jobTitleText":"([^"]+)"')
for url_str in url_strs_list:
    try:
        response_obj = requests.get(url_str, cookies=cookies_dict, headers=headers_dict)
        webpage_html = response_obj.text
        match_obj = job_title_regex.search(webpage_html)
        page_title = eval(f'{{{match_obj.group()}}}')['jobTitleText']
        file_name = re.sub(r'[^A-Za-z0-9]+', ' ', page_title).strip().replace(' ', '_')
        jobListingId = parse_qs(urlparse(url_str).query).get('jobListingId', [''])[0]
        if len(jobListingId):
            file_name = f'{jobListingId}_{file_name}.html'
        else:
            file_name = f'{file_name}.html'
            file_path = os.path.join(cu.SAVES_HTML_FOLDER, file_name)
            if os.path.isfile(file_path):
                file_name = datetime.now().strftime('%Y%m%d%H%M%S%f') + f'_{file_name}'
        file_path = os.path.join(cu.SAVES_HTML_FOLDER, file_name)
        if not os.path.isfile(file_path):
            with open(file_path, 'w', encoding=s.encoding_type) as f:
                print(f'Saving to {file_path}')
                f.write('<html><head><title>')
                f.write(page_title)
                f.write('</title></head><body><div id="jobDescriptionText">')
                match_obj = desc_regex.search(webpage_html)
                html_str = eval(f'{{{match_obj.group()}}}')['job']['description']
                f.write(html_str)
                f.write('</div></body></html>')
            files_list.append(file_name)
        cu.ensure_filename(file_name, verbose=False)
        file_name = cu.clean_text(file_name)
        url_str = cu.clean_text(url_str)
        cypher_str = f'''
            MATCH (fn:FileNames {{file_name: "{file_name}"}})
            SET fn.posting_url = "{url_str}"
            RETURN fn;'''
        with cu.driver.session() as session:
            row_objs_list = session.write_transaction(cu.do_cypher_tx, cypher_str)
            # print(row_objs_list)
    except HTTPError as e:
        print(f'Got an HTTPError with {url_str}: {str(e).strip()}')
    except Exception as e:
        print(f'Got an {e.__class__} error with {url_str}: {str(e).strip()}')

Saving to ../saves/html\1007954692594_Data_Scientist.html
Saving to ../saves/html\1007959629394_Sr_Business_Data_Analyst.html
Saving to ../saves/html\1007947258726_Senior_Data_Analyst.html
Saving to ../saves/html\1007954240607_SR_Financial_Data_Analyst_Fully_remote.html


In [14]:

cu.ensure_navigableparent('END', verbose=False)
for file_name in files_list:
    file_path = os.path.join(cu.SAVES_HTML_FOLDER, file_name)
    page_soup = get_page_soup(file_path)
    row_div_list = page_soup.find_all(name='body')
    for div_soup in row_div_list:
        child_strs_list = ha.get_navigable_children(div_soup, [])
        assert child_strs_list, f'Something is wrong with {file_name}'
        cu.populate_from_child_strings(child_strs_list, file_name, verbose=False)

In [10]:

files_list

['1007957434054_Program_Manager_Data_Analytics_PM22_018999900.html', '1007955267572_Senior_Technical_Product_Manager_Risk_Intelligence.html']